## Model Registry

In [ ]:
from focoos.model_registry import ModelRegistry

registry = ModelRegistry()
print(registry.list_models())


model_info = registry.get_model_info("fai-rtdetr-l-obj365")

model_info.pprint()

## AutoDatasets

In [ ]:
from focoos.data.auto_dataset import AutoDataset
from focoos.data.default_aug import get_default_by_task
from focoos.ports import DatasetLayout, DatasetSplitType, Task

task = Task.DETECTION
layout = DatasetLayout.ROBOFLOW_COCO
auto_datasets = AutoDataset(dataset_name="football", task=task, layout=layout, datasets_root_dir="../data")

train_augs, val_augs = get_default_by_task(task, 640, advanced=False)
train_dataset = auto_datasets.get_split(augs=train_augs.get_augmentations(), split=DatasetSplitType.TRAIN)
valid_dataset = auto_datasets.get_split(augs=val_augs.get_augmentations(), split=DatasetSplitType.VAL)

## Auto Model

In [ ]:
from focoos.auto_model import AutoModel

model = AutoModel.from_pretrained("fai-rtdetr-l-obj365")

# model = AutoModel.from_pretrained("experiments/aquarium2/model_info.json")

## Trainer

In [ ]:
from focoos.ports import TrainerArgs

# from focoos.trainer.trainer import FocoosTrainer

args = TrainerArgs(
    run_name="footballxyz",
    output_dir="./experiments",
    amp_enabled=True,
    batch_size=16,
    max_iters=50,
    eval_period=100,
    learning_rate=0.0001,
    scheduler="MULTISTEP",
    weight_decay=0.0001,
    workers=16,
)


# trainer = FocoosTrainer(args=args, model=model, data_train=train_dataset, data_val=valid_dataset)

# trainer.train()
model.train(args, train_dataset, valid_dataset)

In [ ]:
from pprint import pprint

info = model.model_info
pprint(info)

In [ ]:
from focoos.ports import TrainerArgs

args = TrainerArgs(
    run_name="aquarium2",
    output_dir="./experiments",
    amp_enabled=True,
    batch_size=16,
    max_iters=100,
    eval_period=100,
    learning_rate=0.0001,
    scheduler="MULTISTEP",
    weight_decay=0.0001,
    workers=16,
)

model.test(args, valid_dataset)

In [ ]:
import torch
from PIL import Image

from focoos.auto_model import AutoModel
from focoos.models.fai_rtdetr.processor import RTDetrProcessor

# post_processor = DetectionPostProcessor(valid_dataset.dataset.metadata)
model = AutoModel.from_pretrained("fai-rtdetr-m-coco")
post_processor = RTDetrProcessor()

data = Image.open("../image.jpg")

with torch.no_grad():
    out = model([data])
    out = post_processor.postprocess(out, data)

print(out)